In [ ]:
import numpy as np
from lstchain.image.muon.muon_analysis import pixel_coords_to_telescope
from ctapipe.io import EventSource
import matplotlib.pyplot as plt
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.calib import CameraCalibrator
import os
import re
import numpy as np
from ctapipe.image import tailcuts_clean
from astropy.table import Table, vstack
from ctapipe.visualization import CameraDisplay
import matplotlib.pyplot as plt

import glob
import os
import glob
import re
import numpy as np
from ctapipe.image import tailcuts_clean
import h5py

from ctapipe.visualization import CameraDisplay  # Assuming you're using ctapipe for CameraDisplay


In [ ]:
filename = '/Users/vdk/muons2024//simtel_files/2024year_tuned_nooulier_reflectivity_additional/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 1)
camgeom = source.subarray.tel[1].camera.geometry

tel_description = source.subarray.tels[1]
geom = tel_description.camera.geometry
equivalent_focal_length = tel_description.optics.equivalent_focal_length
x_coord, y_coord = pixel_coords_to_telescope(geom, equivalent_focal_length)
camgeom = source.subarray.tel[1].camera.geometry

In [ ]:
unified_bin_size = 15

def find_alpha_angles(x, y, x_center, y_center, x_cam, y_cam):
    # Calculate the azimuthal angles relative to the ring center
    alpha_angles = np.rad2deg(np.arctan2(y - y_center, x - x_center))
    alpha_angles[alpha_angles < 0] += 360

    # Calculate the reference angle from the ring center to the camera center
    phi0 = np.rad2deg(np.arctan2(y_center - y_cam, x_center - x_cam))
    if phi0 < 0:
        phi0 += 360

    # Adjust angles to have a common zero pointing towards the camera center
    alpha_angles_adjusted = alpha_angles - phi0
    alpha_angles_adjusted[alpha_angles_adjusted < 0] += 360
    return alpha_angles_adjusted

def intensity_binning(x, y, x_center, y_center, image, clean_mask, bin_size=unified_bin_size, x_cam=0, y_cam = 0):
    alpha_angles = find_alpha_angles(x, y, x_center, y_center, x_cam, y_cam)
    num_bins = int(360 / bin_size)
    intensity_azimuth = np.zeros(num_bins)
    image[~clean_mask] = 0

    # Map adjusted angles to bins
    bin_indices = (alpha_angles // bin_size).astype(int) % num_bins
    np.add.at(intensity_azimuth, bin_indices, image)
    return intensity_azimuth

## Simulations

In [ ]:
filename = '/Users/vdk/muons2024/simtel_files/2024year_tuned_nooulier_reflectivity_additional/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 1000)
event_iterator = iter(source)

tel_description = source.subarray.tels[1]
geom = tel_description.camera.geometry
equivalent_focal_length = tel_description.optics.equivalent_focal_length
#x_coord, y_coord = pixel_coords_to_telescope(geom, equivalent_focal_length)

image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray)
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)

intensity_profiles = []
image_profiles = []
image_masks = []

for event in event_iterator:
        calib(event)
        image_processor(event)
        muon_processor(event)
        
        center_x = event.muon.tel[1].ring.center_fov_lon.to_value()
        center_y = event.muon.tel[1].ring.center_fov_lat.to_value()
        if event.muon.tel[1].parameters.completeness > 0.8:
        #if event.muon.tel[1].efficiency.impact.to_value() < 3:
                intensity_profile_temp = intensity_binning(
                        x_coord.degree, 
                        y_coord.degree, 
                        center_x, 
                        center_y, 
                        event.dl1.tel[1].image, 
                        event.dl1.tel[1].image_mask
                )
        
                intensity_profiles.append(intensity_profile_temp)
                image_profiles.append(event.dl1.tel[1].image)
                image_masks.append(event.dl1.tel[1].image_mask)
        if len(intensity_profiles) == 10:
            break

intensity_profiles = np.array(intensity_profiles)
image_profiles = np.array(image_profiles)
        

** TODO ** Make the clean_mask from tailcuts procedure, to have +- 0.4*R region

In [ ]:

id_to_plot = 4

# Data for the first plot
bin_size = unified_bin_size
num_bins = int(360 / bin_size)
bin_edges = np.linspace(0, 360, num_bins + 1)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Create the figure and subplots


# First plot - step plot
for id_to_plot in range(0, 10):
    fig, ax = plt.subplots(1, 3, figsize=(17, 5))  # 1 row, 2 columns
    ax[0].step(bin_centers, intensity_profiles[id_to_plot], where='mid', linewidth=2)
    ax[0].set_xlim(0, 360)
    ax[0].set_ylim(0, 200)
    ax[0].set_xlabel('Azimuthal angle (degrees)')
    ax[0].set_ylabel('Intensity [p.e.]')

    # Second plot - CameraDisplay
    #disp = CameraDisplay(camgeom, image=data_dict[data_event_ids[id_to_plot]], cmap="RdBu", ax=ax[1])
    
    disp = CameraDisplay(camgeom, image=image_profiles[id_to_plot],ax=ax[1])
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    
    disp = CameraDisplay(camgeom, image=image_profiles[id_to_plot],ax=ax[2])
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    disp.highlight_pixels(image_masks[id_to_plot], color='red', linewidth=2)
    #ax[1].set_title("Pixels without signal")

    # Adjust layout
    plt.tight_layout()
    plt.show()

# <center> Data </center>

## <center> Fits files with dl1 images (GlobalpeakWindowSum)</center>

In [ ]:
# Define the FITS file path
fits_file = f'/Users/vdk/muons2024/real_data/17744/fits_tables/tables_run{17744}/muons_LST-1.Run17744.0047.fits'

# Read the FITS file into an Astropy Table
dat = Table.read(fits_file, format='fits')

# Ensure 'good_ring' is of boolean type
dat['good_ring'] = dat['good_ring'].astype(bool)

# Identify one-dimensional and multidimensional columns
one_d_cols = [name for name in dat.colnames if len(dat[name].shape) <= 1]
multi_d_cols = [name for name in dat.colnames if len(dat[name].shape) > 1]

# Convert one-dimensional columns to pandas DataFrame
df = dat[one_d_cols].to_pandas()

# Extract multidimensional data separately
images = dat['image'].tolist() if 'image' in multi_d_cols else None
dist_masks = dat['dist_mask'].tolist() if 'dist_mask' in multi_d_cols else None

# Apply your filtering criteria on the DataFrame
filter_mask = (df['muon_efficiency'] < 100) & (df['size_outside'] < 10000)
df_good_data = df[filter_mask]

# Optionally, filter the multidimensional data based on the same criteria
if images is not None:
    images_good = [img for img, cond in zip(images, filter_mask) if cond]
else:
    images_good = None

if dist_masks is not None:
    dist_masks_good = [mask for mask, cond in zip(dist_masks, filter_mask) if cond]
else:
    dist_masks_good = None

# Check if the lengths match
if images_good is not None and len(images_good) != len(df_good_data):
    raise ValueError("Length of images_good does not match the number of filtered events.")
if dist_masks_good is not None and len(dist_masks_good) != len(df_good_data):
    raise ValueError("Length of dist_masks_good does not match the number of filtered events.")

# Create the dictionary using dictionary comprehension
event_dict = {
    event_id: [
        ring_center_x,
        ring_center_y,
        image if images_good is not None else None,
        dist_mask if dist_masks_good is not None else None
    ]
    for event_id, ring_center_x, ring_center_y, image, dist_mask in zip(
        df_good_data['event_id'],
        df_good_data['ring_center_x'],
        df_good_data['ring_center_y'],
        images_good if images_good is not None else [None] * len(df_good_data),
        dist_masks_good if dist_masks_good is not None else [None] * len(df_good_data)
    )
}

len(event_dict.keys())

In [ ]:
data_intensity_profiles = []
data_event_ids = []
for i, (event_id, data) in enumerate(event_dict.items()):
        center_x = data[0]
        center_y = data[1]
        image = np.array(data[2])
        dist_mask = np.array(data[3])
        if df[df['event_id'] == event_id]['ring_completeness'].iloc[0] > 0.8:     
                intensity_profile_temp = intensity_binning(
                        x_coord.degree, 
                        y_coord.degree, 
                        center_x, 
                        center_y, 
                        image, 
                        dist_mask
                )
        
                data_intensity_profiles.append(intensity_profile_temp)
                data_event_ids.append(event_id)
        if len(data_intensity_profiles) == 10:
            break

data_intensity_profiles = np.array(data_intensity_profiles)
data_event_ids = np.array(data_event_ids)
print(data_intensity_profiles.shape)
print(data_event_ids.shape)

In [ ]:
id_to_plot = 4

# Data for the first plot
bin_size = unified_bin_size
num_bins = int(360 / bin_size)
bin_edges = np.linspace(0, 360, num_bins + 1)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Create the figure and subplots


# First plot - step plot
for id_to_plot in range(0, len(data_intensity_profiles)):   
    fig, ax = plt.subplots(1, 3, figsize=(17, 5))  
    ax[0].step(bin_centers, data_intensity_profiles[id_to_plot], where='mid', linewidth=2)
    ax[0].set_xlim(0, 360)
    ax[0].set_ylim(0, 200)
    ax[0].set_xlabel('Azimuthal angle (degrees)')
    ax[0].set_ylabel('Intensity [p.e.]')

    # Second plot - CameraDisplay
    #disp = CameraDisplay(camgeom, image=data_dict[data_event_ids[id_to_plot]], cmap="RdBu", ax=ax[1])
    
    disp = CameraDisplay(camgeom, image=event_dict[data_event_ids[id_to_plot]][2],ax=ax[1])
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    
    disp = CameraDisplay(camgeom, image=event_dict[data_event_ids[id_to_plot]][2],ax=ax[2])
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    disp.highlight_pixels(event_dict[data_event_ids[id_to_plot]][3], color='red', linewidth=2)

    # Adjust layout
    plt.tight_layout()
    plt.show()

## <center> DL1 images from standard procedure (LocalPeakWindowSum) </center>

In [ ]:
image_folder = glob.glob('/Users/vdk/muons2024/real_data/17744/images/*')

data_dict = {}

for file_path in image_folder:
    filename = os.path.basename(file_path)
    match = re.search(r'image_event(\d+)\.txt$', filename)
    if match:
        event_id = int(match.group(1))
        array_content = np.loadtxt(file_path)
        data_dict[event_id] = array_content
    else:
        print(f"Filename {filename} does not match the expected pattern.")


camgeom = source.subarray.tel[1].camera.geometry
data_mask = {}
for event_id, array_content in data_dict.items():
    mask = tailcuts_clean(camgeom, array_content, picture_thresh=10, boundary_thresh=5)
    data_mask[event_id] = mask

In [ ]:
key_id = 3743987

fig, ax = plt.subplots(1, 2, figsize=(14, 5))

disp = CameraDisplay(camgeom, image=data_dict[key_id],ax=ax[0])
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)

disp = CameraDisplay(camgeom, image=data_dict[key_id],ax=ax[1])
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
disp.highlight_pixels(data_mask[key_id], color='red', linewidth=2)

### <center> Upload all data from one subrun .h5 file for 11 June </center>

-

#### <center> 11 June 2024, Run 17744.0036.h5 </center>

-

In [ ]:
# It can be any dl1 file, from real data or simulations
dl1_file = '/Users/vdk/muons2024/real_data/17744/fits_tables/code_check/dl1_LST-1.Run17744.0036.h5'

dl1_images = []
dl1_mask = []

with h5py.File(dl1_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    for i, dataline in enumerate(data):
        dl1_images.append(dataline[2])
        dl1_mask.append(dataline[4])

dl1_images = np.array(dl1_images)
dl1_mask = np.array(dl1_mask).astype(bool)

In [ ]:
key_id = 1

for key_id in range(0, 3):
    if sum(dl1_images[key_id]) < 10000:
        fig, ax = plt.subplots(1, 2, figsize=(14, 5))

        disp = CameraDisplay(camgeom, image=dl1_images[key_id], ax=ax[0], title=f"Event ID: {key_id}, sum: {sum(dl1_images[key_id]):.2f}")
        disp.cmap = plt.cm.RdBu_r
        disp.title = f"Event ID: {key_id}, sum: {sum(dl1_images[key_id]):.2f}"
        disp.add_colorbar()
        disp.set_limits_percent(95)

        disp = CameraDisplay(camgeom, image=dl1_images[key_id], ax=ax[1])
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        disp.highlight_pixels(dl1_mask[key_id], color='red', linewidth=2)
        plt.tight_layout()
        plt.show()

plt.close()

-

#### <center> 11 June 2024, Run 17744.0035.h5 </center>

-

In [ ]:
# It can be any dl1 file, from real data or simulations
dl1_file = '/Users/vdk/muons2024/real_data/17744/fits_tables/code_check/dl1_LST-1.Run17744.0035.h5'

dl1_images = []
dl1_mask = []

with h5py.File(dl1_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    for i, dataline in enumerate(data):
        dl1_images.append(dataline[2])
        dl1_mask.append(dataline[4])

dl1_images = np.array(dl1_images)
dl1_mask = np.array(dl1_mask).astype(bool)

key_id = 1

for key_id in range(0, 3):
    if sum(dl1_images[key_id]) < 10000:
        fig, ax = plt.subplots(1, 2, figsize=(14, 5))

        disp = CameraDisplay(camgeom, image=dl1_images[key_id], ax=ax[0], title=f"Event ID: {key_id}, sum: {sum(dl1_images[key_id]):.2f}")
        disp.cmap = plt.cm.RdBu_r
        disp.title = f"Event ID: {key_id}, sum: {sum(dl1_images[key_id]):.2f}"
        disp.add_colorbar()
        disp.set_limits_percent(95)

        disp = CameraDisplay(camgeom, image=dl1_images[key_id], ax=ax[1])
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        disp.highlight_pixels(dl1_mask[key_id], color='red', linewidth=2)
        plt.tight_layout()
        plt.show()

plt.close()

-

#### <center> 11 June 2024, Run 17745.0035.h5 </center>

-

In [ ]:
# It can be any dl1 file, from real data or simulations
dl1_file = '/Users/vdk/muons2024/real_data/17744/fits_tables/code_check/dl1_LST-1.Run17745.0035.h5'

dl1_images = []
dl1_mask = []

with h5py.File(dl1_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    for i, dataline in enumerate(data):
        dl1_images.append(dataline[2])
        dl1_mask.append(dataline[4])

dl1_images = np.array(dl1_images)
dl1_mask = np.array(dl1_mask).astype(bool)

key_id = 1

for key_id in range(0, 3):
    if sum(dl1_images[key_id]) < 10000:
        fig, ax = plt.subplots(1, 2, figsize=(14, 5))

        disp = CameraDisplay(camgeom, image=dl1_images[key_id], ax=ax[0], title=f"Event ID: {key_id}, sum: {sum(dl1_images[key_id]):.2f}")
        disp.cmap = plt.cm.RdBu_r
        disp.title = f"Event ID: {key_id}, sum: {sum(dl1_images[key_id]):.2f}"
        disp.add_colorbar()
        disp.set_limits_percent(95)

        disp = CameraDisplay(camgeom, image=dl1_images[key_id], ax=ax[1])
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        disp.highlight_pixels(dl1_mask[key_id], color='red', linewidth=2)
        plt.tight_layout()
        plt.show()

plt.close()

-

#### <center> 11 July 2024 Run 18198.0035.h5 </center>

-

In [ ]:
# It can be any dl1 file, from real data or simulations
dl1_file = '/Users/vdk/muons2024/real_data/17744/fits_tables/code_check/dl1_LST-1.Run18198.0035.h5'

dl1_images = []
dl1_mask = []

with h5py.File(dl1_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    for i, dataline in enumerate(data):
        dl1_images.append(dataline[2])
        dl1_mask.append(dataline[4])

dl1_images = np.array(dl1_images)
dl1_mask = np.array(dl1_mask).astype(bool)

key_id = 1

for key_id in range(0, 3):
    if sum(dl1_images[key_id]) < 10000:
        fig, ax = plt.subplots(1, 2, figsize=(14, 5))

        disp = CameraDisplay(camgeom, image=dl1_images[key_id], ax=ax[0], title=f"Event ID: {key_id}, sum: {sum(dl1_images[key_id]):.2f}")
        disp.cmap = plt.cm.RdBu_r
        disp.title = f"Event ID: {key_id}, sum: {sum(dl1_images[key_id]):.2f}"
        disp.add_colorbar()
        disp.set_limits_percent(95)

        disp = CameraDisplay(camgeom, image=dl1_images[key_id], ax=ax[1])
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        disp.highlight_pixels(dl1_mask[key_id], color='red', linewidth=2)
        plt.tight_layout()
        plt.show()

plt.close()

#### <center> 12 June 2024 Run 17767.0035.h5 </center>

In [ ]:
# It can be any dl1 file, from real data or simulations
dl1_file = '/Users/vdk/muons2024/real_data/17744/fits_tables/code_check/dl1_LST-1.Run17767.0035.h5'

dl1_images = []
dl1_mask = []

with h5py.File(dl1_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    for i, dataline in enumerate(data):
        dl1_images.append(dataline[2])
        dl1_mask.append(dataline[4])

dl1_images = np.array(dl1_images)
dl1_mask = np.array(dl1_mask).astype(bool)

key_id = 1

for key_id in range(0, 3):
    if sum(dl1_images[key_id]) < 10000:
        fig, ax = plt.subplots(1, 2, figsize=(14, 5))

        disp = CameraDisplay(camgeom, image=dl1_images[key_id], ax=ax[0], title=f"Event ID: {key_id}, sum: {sum(dl1_images[key_id]):.2f}")
        disp.cmap = plt.cm.RdBu_r
        disp.title = f"Event ID: {key_id}, sum: {sum(dl1_images[key_id]):.2f}"
        disp.add_colorbar()
        disp.set_limits_percent(95)

        disp = CameraDisplay(camgeom, image=dl1_images[key_id], ax=ax[1])
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        disp.highlight_pixels(dl1_mask[key_id], color='red', linewidth=2)
        plt.tight_layout()
        plt.show()

plt.close()

In [ ]:
disp = CameraDisplay(camgeom, image=images[0])
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)

In [ ]:
disp = CameraDisplay(camgeom, image=images_good[0])
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
disp.highlight_pixels(dist_masks_good[0], color='black', linewidth=2)

In [ ]:
sum(event_dict[data_event_ids[2]][2])

In [ ]:
dist_mask